# Getting Started with TensorFlow
Tensorflow提供了不同层次的API，最底层的API是**Tensorflow Core**，它提供了最完整最全面的API。我们建议机器学习研究人员和那些想要完全掌握模型细节的工程师使用Tensorflow Core。为了简化Tensorflow的使用，Tensorflow还提供了接口简单的高层的API，高层API都建立在Tensorflow Core之上。建议初学者使用这些高层API学习。注意，许多高层API方法包含`contrib`，这意味着这些API还处在开发阶段，处于开发阶段的方法将来可能随时改变甚至被删掉。


我们先学习Tensorflow Core，接下来使用tf.contrib.learn实现相同的模型。只有了解Tensorflow Core才能知道高层API的内部工作方式。


## Tensors

Tensorflow中的核心概念是**tensor**。tensor，也就是张量，维度任意，元素类型相同。用**rank**表示tensor的维度：

In [6]:
3 # 常数的rank是0，shape是[]
[1. ,2., 3.] # rank是1，这是一个向量，shape是[3]
[[1., 2., 3], [4., 5., 6]] # rank是2，这是一个矩阵，shape是[2, 3]
[[[1. ,2., 3.]], [[7., 8., 9.]]] # rank是3，shape是[2, 1, 3]
# rank=前面的[个数


[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]


### Tensorflow Core tutorial

**Importing Tensorflow**

In [14]:
import tensorflow as tf

**The Computational Graph**

你可以把Tensorflow Core程序理解为两个步骤：

1. 创建computational graph

2. 执行computational graph


先来创建一个简单的计算图(computational graph)，每个节点都有>=0个tensor作为输入，和一个输出tensor。有一类节点是constant节点，这类节点不需要输入tensor，输出tensor即节点的存储值。

我们来创建两个浮点tensor:

In [15]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # 默认是tf.float32
print(node1, node2)

(<tf.Tensor 'Const_2:0' shape=() dtype=float32>, <tf.Tensor 'Const_3:0' shape=() dtype=float32>)


注意print并不会输出3.0和4.0的值。相反，输出的是两个节点，只有求值(evaluate)才会产生3.0和4.0。只有在**session**内执行计算图，才能对节点求值。一个session包含了Tensorflow运行时的控制和状态。



接下来创建一个`Session`对象，然后调用`run`方法来执行计算图，对两个节点求值。

In [16]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


Tensorflow中操作运算也是节点，我们来创建更复杂的计算图：

In [17]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ", sess.run(node3))

('node3: ', <tf.Tensor 'Add_1:0' shape=() dtype=float32>)
('sess.run(node3): ', 7.0)


Tensorflow提供一个可视化模块TensorBoard，可以显示计算图。计算图也可以接受外部输入，这时需要使用**placeholdeer**。一个placeholder并不会在创建时提供初始值，而是在以后提供一个值。


In [18]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # + 等同于tf.add(a, b), 是缩写



我们接下来对这个计算图求值，使用feed_dict参数来输入tensor值:


In [19]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


继续添加节点，

In [20]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


机器学习模型都包含必不可少的参数，一般用**Variable**表示模型的参数，

In [21]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b


constant在调用`tf.constant`创建时需要初始化，它的值永远不会改变。相反，variable在调用`tf.Variable'创建时不会执行初始化。如何初始化variable呢？你必须调用一个特殊操作：

In [22]:
init = tf.global_variables_initializer()
sess.run(init)


init是Tensorflow 子图的句柄，对所有的global variable进行初始化，当然了，在调用``sess.run``之前，variable都是未初始化的。

由于x是一个placeholder，我们可以输入不同的x值来同时对linear_model求值:

In [23]:
print(sess.run(linear_model, {x: [1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


 我们已经创建了一个线性模型，但是我们不知道这个模型的性能如何。我们创建一个placeholder:y，提供期望的值，先定义损失函数。

损失函数衡量了模型在训练集上的好坏。我们来创建一个损失函数:

In [24]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1,2,3,4], y:[0,-1,-2,-3]}))

23.66


模型性能不太好，我们重新对W和b赋值，使用tf.assign可以对variable重新赋值:

In [25]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


这里我们直接给出了完美的W和b值，而机器学习的目的就是自动找到最优的模型参数值。接下来我们讲学习如何找到最优参数值。

### tf.train API

详细的讨论机器学习超出了我们tutorial的范畴，只需要知道Tensorflow提供了一系列**optimizer**来改变每个variable使得损失函数值减少。最简单的optimizer是**梯度下降(gradient descent)**, 它根据损失函数对variable的偏导数来改变variable值。


Tensorflow提供了自动求导功能，即使用tf.gradients可以自动对模型求导。为了简化使用，optimizer会自动进行求导。比如：

In [26]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [29]:
sess.run(init) # reset values to incorrent values
for i in range(1000):
    sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})


print(sess.run([W, b])) # 打印出模型的参数值

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


上面的过程就是机器学习的通用流程。

### 完整的程序

完整的训练linear regression model代码:

In [32]:
import tensorflow as tf
import numpy as np

# 模型参数
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
# 模型输入和输出
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
# 损失函数
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
# 训练数据
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values
for i in range(1000):
    sess.run(train, {x:x_train, y:y_train})

# 评估训练准确率
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s" %(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


### tf.contrib.learn

tf.contrib.learn是一个高层Tensorflow库，它简化了机器学习流程，包括：
* running training loops
* running evaluation loops
* managing data sets
* managing feeding

tf.contrib.learn定义了许多常用模型。

#### 基本使用

看看用tf.contrib.learn实现的linear regression model多么简单:

In [33]:
import tensorflow as tf
import numpy as np

# Declare list of features. We only have one real-valued feature.
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

# An estimator is the front end to invoke training (fitting) and evaluation (inference).
# There are many predefined types like linear regression, logistic regression, linear classification,
# logistic classification, and many nerual network classifiers and regressors.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

# Tensorflow provides many helper methods to read and set up data sets.
x = np.array([1., 2., 3., 4.])
y = np.array([0, -1, -2, -3])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4,
                                             num_epochs=1000)

# We can invoke 1000 training step by invoking the 'fit' method and passing the traing data set.
estimator.fit(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
estimator.evaluate(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc8cb207ed0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmplBGKoU/mod

{'global_step': 1000, 'loss': 1.1450492e-11}

#### 创建模型